In [1]:
from pathlib import Path
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.docstore.document import Document
import os
from dotenv import load_dotenv

load_dotenv()

# Definimos las categorías
categorias = {
    "1_poliza_hogar": "Consultas sobre pólizas de hogar",
    "2_siniestralidad": "Consultas sobre siniestralidad",
    "0_no_relevante": "La pregunta no se ajusta a las consultas cubiertas por el bot"
}

def generar_y_guardar_embeddings_categorias():
    # Crear documentos dummy a partir de las categorías
    documents = [
        Document(page_content=text, metadata={"categoria": key})
        for key, text in categorias.items()
    ]

    # Cargar embeddings
    embeddings = OpenAIEmbeddings(openai_api_key=os.getenv("OPENAI_API_KEY"))

    # Crear vectorstore FAISS
    vectorstore = FAISS.from_documents(documents, embeddings)

    # Guardar localmente
    save_dir = Path("src") / "faiss_index"
    vectorstore.save_local(save_dir)
    print(f"✅ Índice FAISS de categorías guardado en {save_dir}")

generar_y_guardar_embeddings_categorias()

✅ Índice FAISS de categorías guardado en src/faiss_index


In [9]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from pathlib import Path
import os
from dotenv import load_dotenv

load_dotenv()

def cargar_y_clasificar(pregunta_usuario):
    # Ruta del índice guardado
    save_dir = Path("src") / "faiss_index"

    # Cargar embeddings y FAISS
    embeddings = OpenAIEmbeddings(openai_api_key=os.getenv("OPENAI_API_KEY"))
    vectorstore = FAISS.load_local(save_dir, embeddings, allow_dangerous_deserialization=True)

    # Buscar la categoría más cercana
    docs = vectorstore.similarity_search(pregunta_usuario, k=1)
    categoria_mas_cercana = docs[0].metadata["categoria"]

    return categoria_mas_cercana, docs

pregunta = "Qué día es hoy?"
resultado, x = cargar_y_clasificar(pregunta)
print("Categoría asignada:", resultado)

Categoría asignada: 2_siniestralidad


In [18]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from pathlib import Path
import numpy as np
import os
from dotenv import load_dotenv

load_dotenv()

def cargar_vectorstore():
    save_dir = Path("src") / "faiss_index"
    embeddings = OpenAIEmbeddings(openai_api_key=os.getenv("OPENAI_API_KEY"))
    vectorstore = FAISS.load_local(save_dir, embeddings, allow_dangerous_deserialization=True)
    return vectorstore

def clasificar_con_umbral(pregunta_usuario, umbral=0.4):
    # Cargar vectorstore
    vectorstore = cargar_vectorstore()

    # Obtener embedding de la pregunta usando embed_query
    q_emb = vectorstore.embedding_function.embed_query(pregunta_usuario)
    q_emb = np.array([q_emb]).astype('float32')  # Convertir a numpy array 2D

    # Buscar los k vecinos más cercanos
    D, I = vectorstore.index.search(q_emb, k=1)  # D: distancias, I: índices

    # Obtener distancia y categoría más cercana
    distancia_min = D[0][0]
    categoria_idx = I[0][0]

    # Obtener nombre de la categoría desde los metadatos
    categoria_mas_cercana = vectorstore.docstore._dict[list(vectorstore.docstore._dict.keys())[categoria_idx]]
    nombre_categoria = categoria_mas_cercana.metadata["categoria"]

    # Decidir si es relevante o no
    if distancia_min < umbral:
        return nombre_categoria, distancia_min
    else:
        return "no_relevante", distancia_min

In [19]:
if __name__ == "__main__":
    preguntas = [
        "¿Cuáles son las coberturas de una póliza de hogar?",
        "¿Cómo declarar un siniestro en mi seguro?",
        "¿Qué día es hoy?",
        "Cómo te llamas?",
        "¿Qué es un seguro de vida?",
        "¿Cuáles son las coberturas incluidas en una póliza de hogar estándar?",
        "¿Qué hacer en caso de robo en casa?",
        "¿Cómo puedo contactar con el servicio de atención al cliente?",
        "¿Qué documentos necesito para contratar un seguro de coche?",
        "¿Cómo puedo modificar mis datos personales en la póliza?",
        "¿Qué es un seguro de responsabilidad civil?",
        "¿Cómo puedo cancelar mi póliza de hogar?",
        "¿Qué hacer si tengo un accidente de tráfico?"
    ]

    for pregunta in preguntas:
        categoria, distancia = clasificar_con_umbral(pregunta)
        print(f"Pregunta: {pregunta}")
        print(f"Categoría asignada: {categoria} | Distancia: {distancia:.4f}\n")

Pregunta: ¿Cuáles son las coberturas de una póliza de hogar?
Categoría asignada: 1_poliza_hogar | Distancia: 0.1506

Pregunta: ¿Cómo declarar un siniestro en mi seguro?
Categoría asignada: 2_siniestralidad | Distancia: 0.3117

Pregunta: ¿Qué día es hoy?
Categoría asignada: no_relevante | Distancia: 0.4675

Pregunta: Cómo te llamas?
Categoría asignada: no_relevante | Distancia: 0.4986

Pregunta: ¿Qué es un seguro de vida?
Categoría asignada: 1_poliza_hogar | Distancia: 0.3365

Pregunta: ¿Cuáles son las coberturas incluidas en una póliza de hogar estándar?
Categoría asignada: 1_poliza_hogar | Distancia: 0.1848

Pregunta: ¿Qué hacer en caso de robo en casa?
Categoría asignada: 1_poliza_hogar | Distancia: 0.3127

Pregunta: ¿Cómo puedo contactar con el servicio de atención al cliente?
Categoría asignada: no_relevante | Distancia: 0.4054

Pregunta: ¿Qué documentos necesito para contratar un seguro de coche?
Categoría asignada: 1_poliza_hogar | Distancia: 0.3108

Pregunta: ¿Cómo puedo modific